In [1]:
%cd WavTokenizer/

/home/h1de0us/audio-eeg-embeddings/WavTokenizer


/home/h1de0us/.pyenv/versions/3.10.13/envs/coursework/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from encoder.utils import convert_audio
import torchaudio
import torch
from decoder.pretrained import WavTokenizer


device=torch.device('cpu')

config_path = "/home/h1de0us/audio-eeg-embeddings/WavTokenizer/configs/wavtokenizer_smalldata_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml"
model_path = "/home/h1de0us/audio-eeg-embeddings/checkpoints/wavtokenizer_medium_music_audio_320_24k_v2.ckpt"


wavtokenizer = WavTokenizer.from_pretrained0802(config_path, model_path)
wavtokenizer = wavtokenizer.to(device)

/home/h1de0us/.pyenv/versions/3.10.13/envs/coursework/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/h1de0us/.pyenv/versions/3.10.13/envs/coursework/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/h1de0us/audio-eeg-embeddings/WavTokenizer/decoder/pretrained.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future releas

In [2]:
eeg_path = '/home/h1de0us/audio-eeg-embeddings/data/song21_Imputed.mat'

import scipy.io
mat = scipy.io.loadmat(eeg_path)
type(mat), mat.keys()

(dict,
 dict_keys(['__header__', '__version__', '__globals__', 'data21', 'fs', 'subs21']))

In [3]:
mat['data21'].shape # timestamp, sr, participants

(125, 34795, 20)

In [5]:
import soundfile as sf
import numpy as np
sr = 34795
sf.write('test.wav', np.ravel(mat['data21'][:, :, 0]), sr)

In [6]:
torchaudio.list_audio_backends()

['soundfile']

In [7]:
wav, sr = torchaudio.load('test.wav')


In [8]:
wav = convert_audio(wav, sr, 24000, 1) 
bandwidth_id = torch.tensor([0])
wav=wav.to(device)
_,discrete_code= wavtokenizer.encode_infer(wav, bandwidth_id=bandwidth_id)
print(discrete_code)

tensor([[[3365,  478,  859,  ..., 3059, 3992, 1312]]])


In [9]:
discrete_code.shape

torch.Size([1, 1, 9375])

In [15]:
# audio_tokens [n_q,1,t]/[n_q,t]
features = wavtokenizer.codes_to_features(discrete_code)
bandwidth_id = torch.tensor([0])  
audio_out = wavtokenizer.decode(features, bandwidth_id=bandwidth_id)

In [16]:
audio_out.shape

torch.Size([1, 3000000])

In [17]:
# timestamp, sr = 125, 24000
# audio_out = audio_out.reshape(timestamp, sr)
# audio_out.shape

In [18]:
from torch.nn import MSELoss

loss = MSELoss()
loss(wav, audio_out)

tensor(1.0778)